# Проектная работа

Заказчик исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. 

**Задача** исследования является подготовить дашборд с итогами опроса с целью определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым». 

Чтобы оценить результаты опроса, оценки обычно делят на три группы: 9-10 баллов — «cторонники» (англ. promoters); 7-8 баллов — «нейтралы» (англ. passives); 0-6 баллов — «критики» (англ. detractors).

**Этапы** работы:

1. Подключимся к базе
2. Подготовим и изучим данные
3. Построим дашборд
4. Создадим презентацию, в которую войдут визуализации, сделанные для дашборда. 

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков». Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.

Интерпретация результатов NPS-опросов исследования может показать, какие клиенты относятся к сторонникам и полезны для бизнеса. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.

# Шаг 1. Получим доступ к базе данных. 

Данные выгрузили в SQLite — СУБД, в которой база данных представлена файлом 'telecomm_csi.db'. 

Чтобы подключиться к базе данных и сохранить данные в датафрейм в pandas, понадобится этот код: 

In [1]:
# Импортируем библиотеки для работы с данными
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = 'telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

У нас имеется 5 таблиц:

- Таблица user
Содержит основную информацию о клиентах. В ней находится важные столбцы, которые помогут объединить данные из других таблиц:

1. location_id - внешний ключ, отсылающий к таблице location
2. age_gr_id - внешний ключ, отсылающий к таблице age_segment
3. tr_gr_id	- внешний ключ, отсылающий к таблице traffic_segment
4. lt_gr_id	- внешний ключ, отсылающий к таблице lifetime_segment

- Таблица location
Справочник территорий, в которых телеком-компания оказывает услуги.

- Таблица age_segment
Данные о возрастных сегментах клиентов.

- Таблица traffic_segment
Данные о выделяемых сегментах по объёму потребляемого трафика.

- Таблица lifetime_segment
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.


Далее по очереди откроем таблички с данными для первичного изучения информации в них.

In [3]:
query = """
SELECT *
FROM user
"""

In [4]:
df = pd.read_sql(query, engine)
df.head(5)

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


In [5]:
query = """
SELECT *
FROM location
"""

In [6]:
df = pd.read_sql(query, engine)
df.head(3)

,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия


In [7]:
query = """
SELECT *
FROM age_segment
"""

In [8]:
df = pd.read_sql(query, engine)
df.head(3)

,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34


In [9]:
query = """
SELECT *
FROM traffic_segment
"""

In [10]:
df = pd.read_sql(query, engine)
df.head(3)

,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1


In [11]:
query = """
SELECT *
FROM lifetime_segment
"""

In [12]:
df = pd.read_sql(query, engine)
df.head(3)

,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3


# Шаг 2. Выгрузка данных
Теперь понятно, как все выглядит внутри этого файла, поэтому приступим к подготовке данных для дашборда. Соберем в одну витрину данные из разных таблиц с помощью SQL-запросов.

In [17]:
# первым делом объединяем таблицы по внешним ключам
# далее указываем столбцы, которые хотим получитт из всех этих таблиц
# для столбцов is_new, gender_segment, nps_group потребуется функция CASE, поскольку нужно нривести данные в другой вид
query = """
SELECT u.user_id, 
       u.lt_day, 
            CASE
                WHEN u.lt_day >= 365 THEN 'нет'
                ELSE 'да'
            END as is_new,
        u.age, 
            CASE
                WHEN u.gender_segment = 1 THEN 'женщина'
                ELSE 'мужчина'
            END AS gender_segment, 
        u.os_name, 
        u.cpe_type_name, 
        loc.country, 
        loc.city, 
        age.title AS age_segment,
        ts.title AS traffic_segment,
        ls.title AS lifetime_segment,
        u.nps_score,
            CASE
                WHEN u.nps_score <= 6 THEN 'критики'
                WHEN u.nps_score >= 9 THEN 'cторонники'
                ELSE 'нейтралы'
            END AS nps_group
FROM user AS u
LEFT JOIN location AS loc ON u.location_id = loc.location_id
LEFT JOIN age_segment AS age ON u.age_gr_id = age.age_gr_id
LEFT JOIN traffic_segment AS ts ON u.tr_gr_id = ts.tr_gr_id
LEFT JOIN lifetime_segment AS ls ON u.lt_gr_id = ls.lt_gr_id
"""

In [18]:
df = pd.read_sql(query, engine)
df.head(20)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,нет,45.0,женщина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,cторонники
1,A001WF,2344,нет,53.0,мужчина,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,cторонники
2,A003Q7,467,нет,57.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,cторонники
3,A004TB,4190,нет,44.0,женщина,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,cторонники
4,A004XT,1163,нет,24.0,мужчина,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,cторонники
5,A005O0,5501,нет,42.0,женщина,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,критики
6,A0061R,1236,нет,45.0,мужчина,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,cторонники
7,A009KS,313,да,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,cторонники
8,A00AES,3238,нет,36.0,женщина,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,cторонники
9,A00F70,4479,нет,54.0,женщина,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,cторонники


Ты верно посмотрела, что в итоге получилось.

Но вообще перед загрузкой данных для дашбордов стоило посмотреть на полученную таблицу поподробнее. Какие значения у нас получились в колонках? Есть ли у нас выбросы, которые нужно как-то обработать перед построением отчетов? Дубликаты?
    
Все необходимые преобразования мы бы потом в Tableau выполнили, но само Tableau - это черный ящик, у нас есть на выходе результат, а преобразований мы не видим. Тут можно было в jupiter-e показать возможные проблемы, которые мы поправим в Tableau прежде чем создавать дашборды.
    
Например, нет ли у нас в NPS значений больше 10? Нет ли у нас в столбце возраст, каких-то необычных показателей? Есть ли выбросы в столбце lt_day? И.т.п.

In [19]:
# Используем эту команду, чтоб сохранить таблицу как CSV-файл.
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Шаг 3. Создание дашборда в Tableau

Загрузили данные в Tableau.

- Создали лист «Города», оформили визуализацию типа map, которая показала количество выступлений по городам России.
- Создали лист «Топ городов» с визуализацией типа horizontal bars, использовали параметр и по нему создали сет, в который вошли топ-10 самых многочисленных по участникам опроса города).
- Создали новый лист «Возраст и пол», оформили таблицу типа highlight table и отобразили количество участников опроса по возрастным и гендерным категориям.
- Создали новый лист «Возраст и пол», оформили визуализацию типа horizontal bars.
- Создали новый лист «Пол клиентов», оформили визуализацию типа horizontal bars и отобразили количество участников опроса по гендерным категориям.
- Создали новый лист «Возраст клиентов», оформили визуализацию типа horizontal bars и отобразили количество участников опроса по возрастным категориям.
- Создали новый лист «Лайфтайм пользователей», оформили визуализацию типа horizontal bars и отобразили количество участников опроса по гендерным категориям.
- Создали новый лист «Дни», оформили визуализацию с распределением участников опроса по количеству дней "жизни" клиента.
- Создали новый лист «Группы одобрения», оформили визуализацию pie chart с двойной осью, чтобы показать процентное соотношение групп NPS, а также создали исчисляемое поле, в котором посчитали общее значение NPS по формуле: % «сторонников» - % «критиков».
- Создали новый лист «Оценки», оформили визуализацию типа horizontal bars и отобразили количество участников опроса по баллам.
- Создали новый лист «Операционные системы и тип устройства», оформили визуализацию типа horizontal bars и отобразили количество участников в каждой категории.
- Создали новый лист «Трафик», оформили визуализацию типа horizontal bars и отобразили количество участников ко количеству используемого трафика.
- Создали дашборд.

# Шаг 4. Создание презентации

С помощью story создали презентацию из пяти слайдов, отвечающих на следующие вопросы:



Опубликовали презентацию на сайте Tableau Public.

Ссылка на дашборд с презентацией:

https://public.tableau.com/views/cityyap21/sheet14_1?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link

# Общий вывод по проекту
Исходя из полученных на опросе данных, можно сделать такие заключения:

*1. Города миллионники возглавляют список городов, чьи жители охотно принимали участие в опросе.  Пользователи из Москвы в подавляющем большинстве активнее всех участвовали в опросе.*

*2. Практически в каждой возрастной группе превалируют участники женского пола, особенно лидирует категория 35-44 лет. Мужчин до 24 лет чуть больше, чем женщин того же возраста.*

*3. К категории "старых" клиентов относится больше половины всех участников опроса, следовательно эти люди продолжают пользоваться услугами данной телекоммуникационной компании уже более 36 мес.*

*4. Лидируют смартфоны и планцеты на базе android, далее следует система IOS.*

*5. Большинство клиентов ТКК потребляет от 1 до 5 пакетов трафика (может измеряться также в битах, байтах, КБ, МБ и т. д.).*

**NPS составляет 21,9% - это хороший показатель, поскольку большинство (52,55%) от общего числа клиентов относятся к сторонникам услуг ТКК**

**Сторонников услуг ТКК можно описать как женщин 35-44 лет, которые в основном пользуются услугами менее 12 мес и потребляют трафик в объеме 1-5 и пользуются смартфонами и планшетами на операционной системе Android.**